In [3]:
pip install folium


Note: you may need to restart the kernel to use updated packages.


In [4]:
import folium
import requests


In [7]:


# Earthquake data from USGS API
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_week.geojson"
response = requests.get(url)
data = response.json()

# Initialize the map
m = folium.Map(location=[48, -102], zoom_start=3)

# Determine marker size based on earthquake magnitude
def marker_size(magnitude):
    if magnitude is not None:
        return magnitude * 5
    else:
        return 5

# Determine marker color based on earthquake depth
def marker_color(depth):
    if depth is not None:
        if depth < 10:
            return 'green'
        elif depth < 30:
            return 'lightgreen'
        elif depth < 50:
            return 'yellow'
        elif depth < 70:
            return 'orange'
        elif depth < 90:
            return 'red'
        else:
            return 'darkred'
    else:
        return 'gray'  # Return gray for unknown depth

# Markers for each earthquake
for feature in data['features']:
    magnitude = feature['properties']['mag']
    depth = feature['geometry']['coordinates'][2]
    location = (feature['geometry']['coordinates'][1], feature['geometry']['coordinates'][0])
    popup_text = f"Magnitude: {magnitude}<br>Depth: {depth} km"
    folium.CircleMarker(location=location, radius=marker_size(magnitude), color=marker_color(depth),
                        fill=True, fill_color=marker_color(depth), fill_opacity=0.7, popup=popup_text).add_to(m)

# legend
legend_html = """
<div style="position: fixed; bottom: 50px; left: 50px; z-index:1000; font-size:14px;">
    <p><i style="background:green; border-radius:50%;"></i> Depth < 10 km</p>
    <p><i style="background:lightgreen; border-radius:50%;"></i> 10 km ≤ Depth < 30 km</p>
    <p><i style="background:yellow; border-radius:50%;"></i> 30 km ≤ Depth < 50 km</p>
    <p><i style="background:orange; border-radius:50%;"></i> 50 km ≤ Depth < 70 km</p>
    <p><i style="background:red; border-radius:50%;"></i> 70 km ≤ Depth < 90 km</p>
    <p><i style="background:darkred; border-radius:50%;"></i> Depth ≥ 90 km</p>
    <p><i style="background:gray; border-radius:50%;"></i> Unknown Depth</p>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map as an image file
m.save("earthquake_map.html")

# Display the map
m
